In [1]:
#Source:   https://www.youtube.com/watch?v=DtBu1u5aBsc&ab_channel=NehaYadav



In [2]:
import sys, os
import pandas as pd
import numpy as np

import cv2
from keras.models import model_from_json
from keras.preprocessing import image

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)

df=pd.read_csv('fer2013.csv')   # the data sets file, contains train/tesr-sets,   i/p is image--> o/p is emotion

#print(df.info())    #file/data info
#print(df["Usage"].value_counts()) # num of trainingset, testset/public test

#print(df.head())   # the head of the columns: emotion & pixels/image




Using TensorFlow backend.
C:\Users\Abouzid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Abouzid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Abouzid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Abouzid\Anaconda3\lib\site-packages\tensorflow\python\framework\d

In [3]:
X_train,train_y,X_test,test_y=[],[],[],[]    #create empty lists for the stes, it will be filled later

for index, row in df.iterrows():       #split the datasets and store them in the lists
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


print(f"X_train.sample.data:{X_train[0:2]}")
print(f"train_y.sample.data:{train_y[0:2]}")
print(f"X_test.sample.data:{X_test[0:2]}")
print(f"test_y.sample.data:{test_y[0:2]}")


X_train.sample.data:[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
train_y.sample.data:[0, 0]
X_test.sample.data:[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
test_y.sample.data:[0, 1]


In [4]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 30
width, height = 48, 48

#convert the lists to arrays with data_type=float32
X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

#cannot produce
#normalizing data between o and 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)
X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

#reshape the data
X_train = X_train.reshape(X_train.shape[0], width, height, 1)
X_test = X_test.reshape(X_test.shape[0], width, height, 1)

# print(f"shape:{X_train.shape}")
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(num_features,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.3))

#3rd convolution layer
model.add(Conv2D(2*num_features, (3, 3), activation='relu'))
model.add(Conv2D(2*num_features, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(2*2*2*2*num_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

#softmax
model.add(Dense(num_labels, activation='softmax'))  #num of emotion=7



# model.summary()

#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

In [3]:

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")             #save weights




Train on 28709 samples, validate on 3589 samples
Epoch 1/30
28709/28709 [==============================] - 192s 7ms/step - loss: 1.7053 - accuracy: 0.3051 - val_loss: 1.5289 - val_accuracy: 0.4018
Epoch 2/30
28709/28709 [==============================] - 186s 6ms/step - loss: 1.5127 - accuracy: 0.4081 - val_loss: 1.4077 - val_accuracy: 0.4567
Epoch 3/30
28709/28709 [==============================] - 190s 7ms/step - loss: 1.4089 - accuracy: 0.4537 - val_loss: 1.3210 - val_accuracy: 0.4909
Epoch 4/30
28709/28709 [==============================] - 185s 6ms/step - loss: 1.3403 - accuracy: 0.4855 - val_loss: 1.3132 - val_accuracy: 0.4957
Epoch 5/30
28709/28709 [==============================] - 185s 6ms/step - loss: 1.2987 - accuracy: 0.4973 - val_loss: 1.2640 - val_accuracy: 0.5082
Epoch 6/30
28709/28709 [==============================] - 190s 7ms/step - loss: 1.2603 - accuracy: 0.5133 - val_loss: 1.2294 - val_accuracy: 0.5364
Epoch 7/30
28709/28709 [==============================] - 198

In [10]:
#use the trained model on cam

cap= cv2.VideoCapture(0)

while True:
    ret, test_im=cap.read() 
    test_img= cv2.flip(test_im, 1)

    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
    faces_detected= face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)
    
    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img, (x,y), (x+w, y+h), (255, 0, 0), thickness= 3)
        roi_gray= gray_img[y:y+w,x:x+h]
        roi_gray= cv2.resize(roi_gray, (48,48))
        img_pixels= image.img_to_array(roi_gray)
        img_pixels= np.expand_dims(img_pixels, axis=0)
        img_pixels/=255     #normalize
        
        predictions= mdl.predict(img_pixels)
        
        #find max indexed array
        max_index= np.argmax(predictions[0])
        emotions= ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion= emotions[max_index]
        
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
        
    resized_img= cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial Emotion Recognition', resized_img)
    
    if cv2.waitKey(10)==ord('q'):
        break
cap.release
cv2.destroyAllWindows()    